In [ ]:
# Development notebook for computational feature selection on CTRPv2 coding genes
# Template for CTRPv2_coding_genes_fractional_RFE.py script
# Cells out of order - REFERENCE ONLY - Intended for documenting rapid iteratation, not sequential reproducibility

In [ ]:
# ENSG - gene
# ENSGR - regulatory
# https://uswest.ensembl.org/info/genome/stable_ids/prefixes.html

In [ ]:
# Devel section

In [ ]:
cell_lines_by_genes = pd.read_csv(
    './data/CCLE_gene_expression_by_cell_line.tsv',
    sep = '\t'
)

In [14]:
# Look at first five ensemble IDs in the raw CCLE object
gene_list = cell_lines_by_genes.columns[2:]
gene_list[0:5]

Index(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460'],
      dtype='object')

In [10]:
# Regulatory genes at end
gene_list

Index(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000000971', 'ENSG00000001036', 'ENSG00000001084',
       'ENSG00000001167',
       ...
       'ENSGR0000237531', 'ENSGR0000237801', 'ENSGR0000263835',
       'ENSGR0000263980', 'ENSGR0000264510', 'ENSGR0000264819',
       'ENSGR0000265350', 'ENSGR0000265658', 'ENSGR0000266731',
       'ENSGR0000270726'],
      dtype='object', length=57820)

In [20]:
len(gene_list)

57820

In [25]:
non_rglrtry_gns = []
rglrtry_gns = []
for gene in gene_list:
#     print(gene)
    if 'ENSGR' in gene:
        rglrtry_gns.append(gene)
        continue
    non_rglrtry_gns.append(gene)
#     break

In [28]:
len(rglrtry_gns)

47

In [ ]:
rglrtry_gns

In [26]:
len(non_rglrtry_gns)

57773

In [29]:
len(rglrtry_gns)+len(non_rglrtry_gns)

57820

In [31]:
# Check for dupes
len(set(non_rglrtry_gns))

57773

In [19]:
'ENSG' in gene

True

In [18]:
'ENSGR' in gene

False

In [ ]:
# HUGO coding genes

In [35]:
# https://www.genenames.org/download/statistics-and-files/
coding = pd.read_csv('data/protein-coding_gene.txt', sep = '\t')

/Users/karlberb/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (31,38,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
len(coding)

19193

In [36]:
coding

,hgnc_id,symbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,...,mamit-trnadb,cd,lncrnadb,enzyme_id,intermediate_filament_db,rna_central_ids,lncipedia,gtrnadb,agr,mane_select
0,HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,19q13.43,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:5,ENST00000263100.8|NM_130786.4
1,HGNC:24086,A1CF,APOBEC1 complementation factor,protein-coding gene,gene with protein product,Approved,10q11.23,10q11.23,ACF|ASP|ACF64|ACF65|APOBEC1CF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:24086,ENST00000373997.8|NM_014576.4
2,HGNC:7,A2M,alpha-2-macroglobulin,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FWP007|S863-7|CPAMD5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:7,ENST00000318602.12|NM_000014.6
3,HGNC:23336,A2ML1,alpha-2-macroglobulin like 1,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FLJ25179|p170,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:23336,ENST00000299698.12|NM_144670.6
4,HGNC:30005,A3GALT2,"alpha 1,3-galactosyltransferase 2",protein-coding gene,gene with protein product,Approved,1p35.1,01p35.1,IGBS3S|IGB3S,iGb3 synthase|isoglobotriaosylceramide synthase,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:30005,ENST00000442999.3|NM_001080438.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19188,HGNC:32058,ZYG11A,"zyg-11 family member A, cell cycle regulator",protein-coding gene,gene with protein product,Approved,1p32.3,01p32.3,ZYG11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:32058,ENST00000371528.2|NM_001004339.3
19189,HGNC:25820,ZYG11B,"zyg-11 family member B, cell cycle regulator",protein-coding gene,gene with protein product,Approved,1p32.3,01p32.3,FLJ13456,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:25820,ENST00000294353.7|NM_024646.3
19190,HGNC:13200,ZYX,zyxin,protein-coding gene,gene with protein product,Approved,7q34,07q34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:13200,ENST00000322764.10|NM_003461.5
19191,HGNC:29027,ZZEF1,zinc finger ZZ-type and EF-hand domain contain...,protein-coding gene,gene with protein product,Approved,17p13.2,17p13.2,KIAA0399|ZZZ4|FLJ10821,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:29027,ENST00000381638.7|NM_015113.4


In [ ]:
# Bio mart coding gene mapping

In [39]:
# http://uswest.ensembl.org/biomart/martview/ec1f98a97c47dcc5eacb06217ed22de8https://uswest.ensembl.org/Multi/Help/Movie?db=core;id=189
# https://uswest.ensembl.org/Multi/Help/Movie?db=core;id=189
# Filters ---> Gene ---> Gene type
biomart = pd.read_csv('data/mart_export.txt', sep = '\t')

In [40]:
biomart

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version
0,ENSG00000198888,ENSG00000198888.2,ENST00000361390,ENST00000361390.2
1,ENSG00000198763,ENSG00000198763.3,ENST00000361453,ENST00000361453.3
2,ENSG00000198804,ENSG00000198804.2,ENST00000361624,ENST00000361624.2
3,ENSG00000198712,ENSG00000198712.1,ENST00000361739,ENST00000361739.1
4,ENSG00000228253,ENSG00000228253.1,ENST00000361851,ENST00000361851.1
...,...,...,...,...
177590,ENSG00000119535,ENSG00000119535.18,ENST00000526980,ENST00000526980.5
177591,ENSG00000119535,ENSG00000119535.18,ENST00000489551,ENST00000489551.5
177592,ENSG00000119535,ENSG00000119535.18,ENST00000533491,ENST00000533491.1
177593,ENSG00000142694,ENSG00000142694.7,ENST00000490466,ENST00000490466.2


In [42]:
len(biomart['Gene stable ID'].unique())

22796

In [43]:
twenty_two_k = list(biomart['Gene stable ID'].unique())

In [45]:
twenty_two_k[-10:] # Map against CCLE expression object

['ENSG00000160678',
 'ENSG00000160679',
 'ENSG00000143553',
 'ENSG00000214193',
 'ENSG00000196182',
 'ENSG00000181817',
 'ENSG00000116885',
 'ENSG00000116898',
 'ENSG00000119535',
 'ENSG00000142694']

In [54]:
coding_genes_CCLE = cell_lines_by_genes[
    cell_lines_by_genes.columns.intersection(
        list(biomart['Gene stable ID'].unique())
    )
]
print('cell lines, genes: ' + str(coding_genes_CCLE.shape))

cell lines, genes: (1165, 19186)


In [55]:
coding_genes_CCLE.to_csv(
    'coding_genes_CCLE.tsv',
    sep = '\t'
)

In [58]:
coding_genes_CCLE

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,...,ENSG00000273213,ENSG00000273217,ENSG00000273238,ENSG00000273259,ENSG00000273269,ENSG00000273274,ENSG00000273291,ENSG00000273294,ENSG00000273331,ENSG00000273398
Cell_line,,,,,,,,,,,,,,,,,,,,,
ACH-000796,3.444932,0.000000,6.898450,1.891419,3.620586,0.014355,0.201634,5.535742,5.070389,4.491212,...,0.070389,0.000000,0.0,0.000000,0.000000,0.000000,0.443607,0.000000,0.000000,0.000000
ACH-000759,5.846493,0.000000,6.369641,4.167519,4.126808,0.070389,3.513491,3.832890,4.655924,4.326969,...,0.000000,0.000000,0.0,0.084064,0.000000,0.137504,3.277985,0.070389,0.000000,0.056584
ACH-000621,3.237258,0.000000,7.982081,2.144046,4.051372,0.111031,1.350497,5.416502,4.090007,4.956521,...,0.925999,0.000000,0.0,0.014355,0.000000,0.831877,0.000000,0.000000,0.000000,0.163499
ACH-000884,4.734981,0.000000,8.087145,2.130931,4.353323,0.084064,0.137504,5.635174,4.668460,4.550285,...,0.000000,0.000000,0.0,0.042644,0.000000,0.000000,0.056584,0.847997,0.000000,0.056584
ACH-000876,4.204767,0.000000,6.255123,2.000000,2.367371,0.014355,2.157044,6.296457,5.960002,4.038261,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.321928,0.765535,0.097611,0.097611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000022,4.220330,0.000000,6.383359,2.718088,3.023255,0.014355,0.084064,5.780310,5.537296,4.596339,...,0.000000,0.000000,0.0,0.000000,0.042644,0.014355,0.028569,0.575312,0.000000,0.111031
ACH-000655,4.772414,0.000000,6.250014,1.803227,3.427606,0.028569,1.000000,5.723012,3.467279,3.633431,...,1.432959,0.000000,0.0,0.000000,0.056584,0.176323,0.056584,0.000000,0.000000,0.056584
ACH-000091,2.803227,0.070389,6.342875,2.021480,2.580145,0.214125,1.704872,5.331275,5.631977,3.972693,...,0.111031,0.000000,0.0,0.097611,0.000000,0.536053,0.250962,0.189034,0.823749,0.137504


In [59]:
coding_genes_CCLE_read = pd.read_csv(
    'coding_genes_CCLE.tsv',
    sep = '\t',
    index_col = 0
)

In [57]:
coding_genes_CCLE_read.head()

,Cell_line,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000273213,ENSG00000273217,ENSG00000273238,ENSG00000273259,ENSG00000273269,ENSG00000273274,ENSG00000273291,ENSG00000273294,ENSG00000273331,ENSG00000273398
0,ACH-000796,3.444932,0.0,6.898450,1.891419,3.620586,0.014355,0.201634,5.535742,5.070389,...,0.070389,0.0,0.0,0.000000,0.0,0.000000,0.443607,0.000000,0.000000,0.000000
1,ACH-000759,5.846493,0.0,6.369641,4.167519,4.126808,0.070389,3.513491,3.832890,4.655924,...,0.000000,0.0,0.0,0.084064,0.0,0.137504,3.277985,0.070389,0.000000,0.056584
2,ACH-000621,3.237258,0.0,7.982081,2.144046,4.051372,0.111031,1.350497,5.416502,4.090007,...,0.925999,0.0,0.0,0.014355,0.0,0.831877,0.000000,0.000000,0.000000,0.163499
3,ACH-000884,4.734981,0.0,8.087145,2.130931,4.353323,0.084064,0.137504,5.635174,4.668460,...,0.000000,0.0,0.0,0.042644,0.0,0.000000,0.056584,0.847997,0.000000,0.056584
4,ACH-000876,4.204767,0.0,6.255123,2.000000,2.367371,0.014355,2.157044,6.296457,5.960002,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,1.321928,0.765535,0.097611,0.097611


In [79]:
# Fractional RFE
# from drg_rsp_db_row_lables.ipynb in v2 directory

import pandas as pd
import time
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
import argparse

# parser = argparse.ArgumentParser(
#     description = 'Drug index'
# )

# parser.add_argument(
#     'drug_frame_index',
#     type = int,
#     help = 'Input drug frame index number'
# )

# args = parser.parse_args()

dummy_loop = [0, 1, 3]

for i in dummy_loop:

    drug_frame_index = i
    # drug_frame_index = args.drug_frame_index

    file_read_start = time.time()

#     coding_genes_CCLE = pd.read_csv(
#         'coding_genes_CCLE.tsv',
#         sep = '\t',
#         index_col = 0
#     )
    print('gene file read')

    gexp_cell_lines = coding_genes_CCLE.index

    CTRPv2 = pd.read_csv(
        './data/drug_response_CTRPv2.tsv',
        sep = '\t',
        index_col = 'Cell_line'
    )

    CTRPv2_unique_compounds = CTRPv2.Drug.unique()
    
    multiple_responses = []
    for drg in CTRPv2_unique_compounds:
        drug_frame = CTRPv2[ 
            CTRPv2.Drug == drg
        ]
        
        if len(drug_frame.index.unique()) != len(drug_frame):
            multiple_responses.append(drg)
    
    CTRPv2 = CTRPv2[
        CTRPv2.Drug.isin(
            multiple_responses
        ) == False
    ]
    
    CTRPv2_unique_compounds = CTRPv2.Drug.unique()
    
    compound = CTRPv2_unique_compounds[
        drug_frame_index
    ]
    
    drug_frame = CTRPv2[ 
        CTRPv2.Drug == CTRPv2_unique_compounds[
            drug_frame_index
        ]
    ]
    
    drug_frame_cell_lines = drug_frame.index
    
    cell_line_intersection = gexp_cell_lines.intersection(
        drug_frame_cell_lines
    )
    
    X = coding_genes_CCLE[
        coding_genes_CCLE.index.isin(
            cell_line_intersection
        )
    ].iloc[:, 2:]
    
#     del(coding_genes_CCLE)
    
    # Devel object:
#     X_cut = X.iloc[:, :5000]
    
    y = drug_frame[
        drug_frame.index.isin(
            cell_line_intersection
        )
    ].Response
    
    estimator = SVR(kernel="linear")
    RFE_start_time = time.time()
    selector = RFE(
        estimator,
        n_features_to_select=10,
        step=.5 # Cut 50% per step
    )
    
    RFE_step = '50%'
    
    selector = selector.fit(X, y)    
#     selector = selector.fit(X_cut, y)

    out = pd.concat(
        [drug_frame[
            drug_frame.index.isin(
                cell_line_intersection
                )
            ],
#         X_cut.loc[         
        X.loc[
            :, selector.support_
            ]
        ], axis = 1
    )
    
    end_time = time.time() - RFE_start_time
    
    out.to_csv(
        'CTRP_features/selected_features/selected_features_' + compound + '_' + str(
            round(
                end_time, 2
            )
        ) + '_' + RFE_step + '_step.tsv',
        sep = '\t',
    )
    
    print('done')

gene file read
done
gene file read
done
gene file read
done


In [78]:
compound

'piperlongumine'

In [76]:
out.head()

,Drug,Response,ENSG00000017483,ENSG00000051523,ENSG00000078399,ENSG00000089820,ENSG00000099399,ENSG00000105388,ENSG00000109255,ENSG00000113140,ENSG00000115590,ENSG00000116962
Cell_line,,,,,,,,,,,,
ACH-000464,BRD4132,0.113025,0.378512,0.495695,0.687061,2.153805,0.014355,0.250962,0.575312,9.939976,0.176323,4.465322
ACH-000028,BRD4132,0.015200,0.356144,9.206624,0.042644,4.811471,0.042644,1.863938,2.100978,1.250962,0.111031,0.084064
ACH-000351,BRD4132,0.124864,1.084064,7.818518,0.137504,1.594549,0.056584,0.263034,5.857483,10.845490,0.250962,1.090853
ACH-000026,BRD4132,0.200045,0.847997,8.247405,7.438542,1.981853,0.042644,0.097611,0.163499,7.860156,0.000000,2.959770
ACH-000792,BRD4132,0.130551,3.452859,7.056800,0.584963,1.655352,0.111031,0.189034,2.910733,11.566530,0.226509,5.759422
...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000160,BRD4132,0.000000,0.389567,0.356144,0.028569,0.400538,0.000000,0.042644,4.612352,1.422233,0.713696,0.056584
ACH-001020,BRD4132,0.002086,0.201634,0.226509,0.000000,4.971773,0.000000,0.000000,3.405992,0.495695,0.000000,0.000000
ACH-001275,BRD4132,0.037387,0.310340,7.756023,3.346248,1.384050,0.000000,0.000000,1.851999,5.544424,0.189034,6.825531


In [ ]:
# To Exacloud .py script

In [ ]:
# Final conversion notes for iteration 3.1:
# Remove dummy loop
# Write de-duped CTRP file deleting the 17 drugs
# Turn down RFE cut % to 25
# Turn up selected features to 20

In [80]:
# Index range for Sbatch array: 0-489:1
len(CTRPv2_unique_compounds)

490

In [ ]:
import pandas as pd
import time
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
import argparse

parser = argparse.ArgumentParser(
    description = 'Drug index'
)

parser.add_argument(
    'drug_frame_index',
    type = int,
    help = 'Input drug frame index number'
)

args = parser.parse_args()

dummy_loop = [0]

for i in dummy_loop:

    drug_frame_index = args.drug_frame_index

    file_read_start = time.time()

    coding_genes_CCLE = pd.read_csv(
        './data/coding_genes_CCLE.tsv',
        sep = '\t',
        index_col = 0
    )

    gexp_cell_lines = coding_genes_CCLE.index

    CTRPv2 = pd.read_csv(
        './data/drug_response_CTRPv2.tsv',
        sep = '\t',
        index_col = 'Cell_line'
    )

    CTRPv2_unique_compounds = CTRPv2.Drug.unique()
    
    multiple_responses = []
    for drg in CTRPv2_unique_compounds:
        drug_frame = CTRPv2[ 
            CTRPv2.Drug == drg
        ]
        
        if len(drug_frame.index.unique()) != len(drug_frame):
            multiple_responses.append(drg)
    
    CTRPv2 = CTRPv2[
        CTRPv2.Drug.isin(
            multiple_responses
        ) == False
    ]
    
    CTRPv2_unique_compounds = CTRPv2.Drug.unique()
    
    compound = CTRPv2_unique_compounds[
        drug_frame_index
    ]
    
    drug_frame = CTRPv2[ 
        CTRPv2.Drug == CTRPv2_unique_compounds[
            drug_frame_index
        ]
    ]
    
    drug_frame_cell_lines = drug_frame.index
    
    cell_line_intersection = gexp_cell_lines.intersection(
        drug_frame_cell_lines
    )
    
    X = coding_genes_CCLE[
        coding_genes_CCLE.index.isin(
            cell_line_intersection
        )
    ].iloc[:, 2:]
    
    del(coding_genes_CCLE)
    
    y = drug_frame[
        drug_frame.index.isin(
            cell_line_intersection
        )
    ].Response
    
    estimator = SVR(kernel="linear")
    RFE_start_time = time.time()
    selector = RFE(
        estimator,
        n_features_to_select=10,
        step=.5
    )
    
    RFE_step = '50%'
    
    selector = selector.fit(X, y)    

    out = pd.concat(
        [drug_frame[
            drug_frame.index.isin(
                cell_line_intersection
                )
            ],      
        X.loc[
            :, selector.support_
            ]
        ], axis = 1
    )
    
    end_time = time.time() - RFE_start_time
    
    out.to_csv(
        'CTRP_features/selected_features/selected_features_' + compound + '_' + str(
            round(
                end_time, 2
            )
        ) + '_' + RFE_step + '_step.tsv',
        sep = '\t',
    )
    
    print('done')